In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch.nn as nn
from torch.autograd import Variable
import torch
#import bcolz
import time
from utils import *
from sklearn.svm import LinearSVC



cwd = os.getcwd()
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

prep1 = transforms.Compose([
            #transforms.RandomSizedCrop(512),
            #transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])

type = 2 #['2', '5', 'all'] #trial types - #num of classes to test



transformer = transforms.Compose([transforms.ToTensor(),
                                 transforms.Normalize((0.5,),(1.0,))])


train = torchvision.datasets.ImageFolder(cwd + '/Samples2/train',transform=transformer)
train_loader = torch.utils.data.DataLoader(train,
                                          batch_size=48,
                                          shuffle=False,
                                          num_workers=8)

test = torchvision.datasets.ImageFolder(cwd + '/Samples2/test',transform=transformer)
test_loader = torch.utils.data.DataLoader(test,
                                          batch_size=48,
                                          shuffle=False,
                                          num_workers=8)    

use_gpu = torch.cuda.is_available()


device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model_conv = torchvision.models.resnet50(pretrained=True)

#model_conv = models.alexnet(pretrained=True)
#model_conv = models.squeezenet1_0(pretrained=True)
#model_conv = models.vgg16(pretrained=True)
#model_conv = models.densenet161(pretrained=True)
#model_conv = models.inception_v3(pretrained=True)

for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features


class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
    
model_conv.fc = Identity()
#print(model)

model= model_conv.to(device)

torch.cuda.is_available()
torch.cuda.set_device(1)
torch.backends.cudnn.benchmark=True
torch.backends.cudnn.fastest = True


optimizer = optim.Adam(model.parameters(), lr=0.00001)

criterion = nn.CrossEntropyLoss()

def feat_extr(loader):
    out =  Variable(torch.FloatTensor())
    labels =  Variable(torch.FloatTensor())
    image = 0
    count = 0
    batch = 0
    for batch_idx, (x, target) in enumerate(loader):
        if x.shape[0] != 48:
            break
        for i in range(x.shape[0]):
            if count == 0:
                image = x[i,2:,:,:]
                count += 1
                continue
            elif count == 1:
                image = torch.cat((image, x[i,2:,:,:]), dim=0)
                count += 1
                continue
            else:
                image = torch.cat((image, x[i,2:,:,:]), dim=0)
                count = 0
            if i == 2:
                batch = image.reshape((-1,3,512,512))
            else:
                batch = torch.cat((batch, image.reshape((1,3,512,512))), dim=0)
                #print(batch.shape)
        tar = []
        for i in range(target.shape[0]):
            if i % 3 == 0:
                tar.append(target[i].item())
        target = torch.LongTensor(tar)
        x = batch.cuda()
        if len(out.size()) == 1:
            out = model(x)
            labels = target
        else:
            #print(out.shape)
            #print(labels.shape)
            out = torch.cat((out, model(x.cuda())), dim=0)
            labels = torch.cat((labels, target), dim=0)
        
    print(out.shape)
    return out, labels   
out, labels = feat_extr(train_loader)
correct_cnt, ave_loss = 0, 0
total_cnt = 0



torch.Size([4176, 204800])


In [22]:

clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(out, labels)

out, labels = feat_extr(test_loader)
preds = []
for i in out:
    preds.append(clf.predict(i.reshape(1,-1)).reshape(-1))


torch.Size([1040, 204800])


In [20]:
count = 0
TP = 0
FN = 0
FP = 0

for indx in range(labels.shape[0]):  
    if labels[indx].item() > 0.5:
        if preds[indx][0] > 0.5:
            TP += 1
        else:
            FN += 1
    else:
        if preds[indx][0] > 0.5:
            FP += 1

    count += 1
                    
if TP == 0 and FP == 0:
    FP = 1
if TP == 0 and FN == 0:
    FN = 1
precision = TP / (TP + FP)
recall = TP / (TP + FN)
if precision == 0 and recall == 0:
    recall = 1
f1 = 2*((precision*recall)/(precision+recall))

print('Accuracy')
print(f1)

0
Accuracy
0.8769841269841269
